In [1]:
!git clone https://github.com/zhangbo2008/opensource_tts_chinese_and_english

Cloning into 'opensource_tts_chinese_and_english'...
remote: Enumerating objects: 2640, done.
remote: Total 2640 (delta 0), reused 0 (delta 0), pack-reused 2640
Receiving objects: 100% (2640/2640), 54.93 MiB | 27.03 MiB/s, done.
Resolving deltas: 100% (971/971), done.


In [2]:
%cd opensource_tts_chinese_and_english

/kaggle/working/opensource_tts_chinese_and_english


In [3]:
!mv  transformers transformers9999

In [4]:
%ls

11.wav                   666/            chinese_tts2.py
2.mp3                    8888.py         detr_model_debug.py
2021-06-16-16-29-56.wav  ChineseASR.py   english_tts.py
2021-07-01-17-42-26.wav  EnglishASR.py   readme.md
2222222.py               TensorFlowTTS/  tensorflow_tts/
22English.wav            __init__.py     test.tsv
3333333.py               audio.wav       transformers9999/
444.py                   chinese.wav     vit_model_debug.py
444的语音文件.flac       chinese_asr.py  刷时间.py
65555.py                 chinese_tts.py  语音识别中文字典


In [5]:
!pip install tensorflow==2.12.0


In [6]:
!pip install jamo inflect pypinyin  tensorflow_addons  g2p_en


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16258 sha256=4820b36ff8d42b14567aa033d7378d573cd4e55eb055ac26724d59fb063ff982
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [7]:
!pip install transformers

In [12]:
!pip install librosa==0.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.9.2
    Uninstalling librosa-0.9.2:
      Successfully uninstalled librosa-0.9.2


In [14]:

import librosa
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
# 配置好cache,不然就跑c盘去了.c盘太宝贵了555555555
processor = Wav2Vec2Processor.from_pretrained("ydshieh/wav2vec2-large-xlsr-53-chinese-zh-cn-gpt")
tokenizer = processor
model = Wav2Vec2ForCTC.from_pretrained("ydshieh/wav2vec2-large-xlsr-53-chinese-zh-cn-gpt")

resampler = torchaudio.transforms.Resample(48_000, 16_000)
if 1:


    # Preprocessing the datasets.
    # We need to read the aduio files as arrays
    def speech_file_to_array_fn(batch):
        speech_array, sampling_rate = torchaudio.load(batch["path"])
        batch["speech"] = resampler(speech_array).squeeze().numpy()
        return batch
    #========run demo
    # test_dataset = test_dataset.map(speech_file_to_array_fn)
    import soundfile as sf
    name='chinese.wav'
    src_sig, sr = sf.read(name)  # name是要 输入的wav 返回 src_sig:音频数据  sr:原采样频率
    dst_sig = librosa.resample(src_sig, orig_sr=sr, target_sr=16000)
    inputs = processor(dst_sig, sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

    predicted_ids = torch.argmax(logits, dim=-1)

    print("Prediction:", processor.batch_decode(predicted_ids))
    print("答案",'宋朝末年年间定居粉岭围')
    # print("Reference:", test_dataset[:2]["sentence"])



#=========run valid


#
# test_dataset = load_dataset("common_voice", "zh-CN", split="test",cache_dir='E:/dataForVoice')
# savepath=test_dataset['path'][0]
# savesentence=test_dataset['sentence'][0]
# print(1)
#
#
# resampler = torchaudio.transforms.Resample(48_000, 16_000)
#
# # Preprocessing the datasets.
# # We need to read the aduio files as arrays
# def speech_file_to_array_fn(batch):
#     speech_array, sampling_rate = torchaudio.load(batch["path"])
#     batch["speech"] = resampler(speech_array).squeeze().numpy()
#     return batch
#
#
# # test_dataset = test_dataset.map(speech_file_to_array_fn)
#
# aaa=      resampler(  torchaudio.load(savepath)[0]  ).squeeze().numpy()
# # test_dataset = test_dataset.map(speech_file_to_array_fn)
# inputs = processor(aaa, sampling_rate=16_000, return_tensors="pt", padding=True)
#
# with torch.no_grad():
#     logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
#
# predicted_ids = torch.argmax(logits, dim=-1)
#
# print("Prediction:", processor.batch_decode(predicted_ids))
# print("Reference:", savesentence)
# #======================train   下面是重要的训练代码. 进行finetune.



if 0:

    epoch=10


    # 2021-06-05,16点06 思路是从英文直接超过来试试.
    import soundfile as sf
    name='chinese.wav'
    src_sig, sr = sf.read(name)  # name是要 输入的wav 返回 src_sig:音频数据  sr:原采样频率
    dst_sig = librosa.resample(src_sig, sr, 16000)
    input_values = processor(dst_sig, sampling_rate=16_000, return_tensors="pt", padding=True).input_values





    # compute loss
    target_transcription = "宋朝末年年间定居粉岭围"
    processor.tokenizer.save_vocabulary("保存中文语音识别的字典")
    # wrap processor as target processor to encode labels
    with processor.as_target_processor():
        labels = processor(target_transcription, return_tensors="pt").input_ids       # 把答案也进行编码,跟语音编码是一样的.上面用的特征提取器, 下面用的nlp编码.
    # 编码之后也是38这个长度.




    from transformers import AutoTokenizer, AutoModelWithLMHead, AdamW

    class A():
        pass
    args=A()
    args.learning_rate=3e-5
    args.adam_epsilon=1e-8
    args.weight_decay=0
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    # 开启finetune模式 ,,,,,,,C:\Users\Administrator\.PyCharm2019.3\system\remote_sources\-456540730\-337502517\transformers\data\processors\squad.py 从这个里面进行抄代码即可.
    model.zero_grad()
    model.train()

    print('start_train')
    for _ in range(epoch):




        loss = model(input_values, labels=labels).loss
        loss.backward()
        optimizer.step()

        model.zero_grad()
        print(loss)
    print("train_over!!!!!!!!!!")







Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prediction: ['宋朝末年年间定区分命为']
答案 宋朝末年年间定居粉岭围


In [15]:
from IPython.display import Audio


Audio(audio.numpy(), rate=22050)



NameError: name 'audio' is not defined